In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
repo_dir = '/home/labs/amit/noamsh/repos/MM_2023'
sys.path.append(repo_dir)

In [3]:
from pathlib import Path
from datetime import date
from omegaconf import OmegaConf

import scanpy as sc


In [4]:
config_path = Path(repo_dir, 'config.yaml')
conf = OmegaConf.load(config_path)

In [5]:
ts_iso = date.today().isoformat()
figures_dir = Path(conf.outputs.output_dir, "figures", ts_iso)

### 

In [11]:
from pp_adata import pre_process
from train_scvi_model import norm_and_hvg
from infer_annotation import infer_annotation_of_unlabeled_cells

In [8]:
conf.pp.min_num_genes_per_cell = 500
bad_SPID_plates_sb = set([f"SB{i}" for i in range(190, 220)])

pp_adata = pre_process(conf)
good_spid_sb_mask = (pp_adata.obs['Seq.Batch.ID'].apply(lambda x: x not in bad_SPID_plates_sb)) & (pp_adata.obs['Method'] == "SPID")
pp_adata = pp_adata[good_spid_sb_mask | (pp_adata.obs['Method'] == "MARS")]

good_cells_adata = norm_and_hvg(pp_adata, conf)
good_cells_adata

/home/labs/amit/noamsh/.conda/envs/CLL_2024/lib/python3.9/site-packages/scanpy/preprocessing/_simple.py:275: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var["n_cells"] = number
/home/labs/amit/noamsh/.conda/envs/CLL_2024/lib/python3.9/site-packages/anndata/_core/anndata.py:1301: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/labs/amit/noamsh/.conda/envs/CLL_2024/lib/python3.9/site-packages/anndata/_core/anndata.py:1301: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/labs/amit/noamsh/.conda/envs/CLL_2024/lib/python3.9/site-packages/anndata/_core/anndata.py:1301: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/labs/amit/noamsh/repos/MM_2023/train_scvi_model.py:47: ImplicitModificationWarning: Setting elemen

AnnData object with n_obs × n_vars = 187019 × 3862
    obs: 'Amp.Batch.ID', 'Seq.Batch.ID', 'Batch.Set.ID', 'Included', 'Tissue', 'Gating', 'Total.PC', 'Total.NonPC', 'Initial.Code', 'Disease', 'Time', 'Project', 'Cohort', 'Weizmann.Code', 'Hospital.Code', 'Clinical.Data', 'Method', 'cID', 'super_Population', 'Populations', 'n_genes', 'n_counts', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
    var: 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
    uns: 'log1p'
    layers: 'counts'

In [ ]:
good_cells_adata_with_annotaion = infer_annotation_of_unlabeled_cells(conf, on_adata=good_cells_adata)
good_cells_adata_with_annotaion

/home/labs/amit/noamsh/.conda/envs/CLL_2024/lib/python3.9/abc.py:119: FutureWarning: SparseDataset is deprecated and will be removed in late 2024. It has been replaced by the public classes CSRDataset and CSCDataset.

For instance checks, use `isinstance(X, (anndata.experimental.CSRDataset, anndata.experimental.CSCDataset))` instead.

For creation, use `anndata.experimental.sparse_dataset(X)` instead.

  return _abc_instancecheck(cls, instance)


INFO     File                                                                                                      
         /home/labs/amit/noamsh/repos/MM_2023/outputs/scvi_model_MARS_SPID_combined_genes_ts_2024-05-21/model.pt   
         already downloaded                                                                                        


/home/labs/amit/noamsh/.conda/envs/CLL_2024/lib/python3.9/abc.py:119: FutureWarning: SparseDataset is deprecated and will be removed in late 2024. It has been replaced by the public classes CSRDataset and CSCDataset.

For instance checks, use `isinstance(X, (anndata.experimental.CSRDataset, anndata.experimental.CSCDataset))` instead.

For creation, use `anndata.experimental.sparse_dataset(X)` instead.

  return _abc_instancecheck(cls, instance)
/home/labs/amit/noamsh/repos/MM_2023/infer_annotation.py:54: FutureWarning: In the future, the default backend for leiden will be igraph instead of leidenalg.

 To achieve the future defaults please pass: flavor="igraph" and n_iterations=2.  directed must also be False to work with igraph's implementation.
  sc.tl.leiden(adata, resolution=res, neighbors_key=neighborhood_key)


In [ ]:
sc.pl.umap(good_cells_adata_with_annotaion, color=)

###

In [ ]:
from sklearn.model_selection import train_test_split
cell_train, cell_test = train_test_split(good_cells_adata_with_annotaion.obs_names)
for sample_proprotion in [0.]